In [1]:
import mdtraj as md

In [2]:
dirs = '/home/adria/perdiux/prod/lammps/dignon/hnRPA/3RE-T-hnRPA/xtc_temper0.xtc'
topo = '/home/adria/perdiux/prod/lammps/dignon/hnRPA/3RE-T-hnRPA/hps_trj.pdb'
md.load(dirs,top=topo)

RuntimeError: XTC read error: Compressed 3d coordinate

In [3]:
dirs = '/home/adria/perdiux/prod/lammps/dignon/hnRPA/RE-T-hnRPA/xtc_temper0.xtc'
topo = '/home/adria/perdiux/prod/lammps/dignon/hnRPA/RE-T-hnRPA/hps_trj.pdb'
md.load(dirs,top=topo)

<mdtraj.Trajectory with 20001 frames, 152 atoms, 1 residues, and unitcells at 0x7fe818f1eb10>